## Imports

In [50]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv

## Read data

#### *****TODO: simulate appropriate data that would mimic animals behavior underwater

#### Generate Time 

In [7]:
t_final = 30
dtAccel = 0.005  # TODO: assumption that the time interval is constant 
t_step = np.arange(dtAccel,t_final,dtAccel)
print(len(t_step))
print(t_step)

5999
[5.0000e-03 1.0000e-02 1.5000e-02 ... 2.9985e+01 2.9990e+01 2.9995e+01]


#### True Accelerometer for comparison
###### (TODO: ground truth from simulator)

In [8]:
true_accel = np.concatenate(([0],np.random.random_sample((len(t_step),))*10))
print(true_accel)
print(len(true_accel))

[0.         8.73509934 3.78928482 ... 7.70730564 1.76744331 0.84112238]
6000


#### True velocity and acceleration for comparison

In [12]:
#a priori statistics => x ~ N(0,10^2), v ~ N(100,1^2)
pos_0 = 0 # meters
vel_0 = 100 # meters/second 


true_vel = [vel_0]
true_pos = [pos_0]

for i in range(len(t_step)):
    if i == 0:
        p_last = pos_0
        v_last = vel_0
        
    v_current = v_last + (true_accel[i]*dtAccel)
    true_vel.append(v_current)
    v_last = v_current
    
    p_current = p_last + v_last*dtAccel + true_accel[i]*(dtAccel**2)/2
    true_pos.append(p_current)
    p_last = p_current
    
print("acceleration")
print(len(true_accel))
print(true_accel[0:10])

print("velocity")


print(len(true_vel))
print(true_vel[0:10])

print("position")

print(len(true_pos))
print(true_pos[0:10])

acceleration
6000
[0.         8.73509934 3.78928482 6.13034805 0.25108571 7.43077304
 0.19721144 5.29184928 9.86679845 7.65232885]
velocity
6000
[100, 100.0, 100.04367549668083, 100.0626219207878, 100.09327366103686, 100.09452908958752, 100.13168295479747, 100.13266901197944, 100.1591282583695, 100.2084622506021]
position
6000
[0, 0.5, 1.0003275662251063, 1.5006880418893127, 2.00123103954512, 2.501706823564434, 3.002458123001446, 3.5031239332042987, 4.003985722612121, 4.505151368845714]


#### Generating Acceleration, velocity, and position  with bias and gaussian noise

In [51]:
# white gaussian noise with zero mean and variance = 0.0004
wV_Accel = .0004 # Accelerometer white noise variance 
wm_Accel = 0 # Accelerometer white noise mean
wN_Accel = np.array([wm_Accel, wV_Accel])

# bias with a priori statistics b_a ~ N(0,0.01)
bu_Accel = 0 # Mean of bias term
bV_Accel = .01 # Variance of bias (m/s^2)^2
bN_Accel = np.array([bu_Accel, bV_Accel])

print("white gaussian noise of accelerometer : ", wN_Accel)
print("bias of accelerometer: ",bN_Accel)


def get_white_noise_accelerometer():
    return np.asscalar(np.random.normal(wN_Accel[0],np.sqrt(wN_Accel[1]),1))

def set_bias_accelerometer():
    return np.asscalar(np.random.normal(bN_Accel[0], np.sqrt(bN_Accel[1]),1))

# a = a + w + b
def get_accelerometer_reading(i, white_noise):
    return true_accel[i] + white_noise + bu_Accel

# Takes in last state @ time = k, calculates the new acceleration state @ time = k+1
def get_pos_vel_from_accel(ilast,xlast,white_noise):
    # v_new = a_last*dt + v_last
    vAc = get_accelerometer_reading(ilast,white_noise)*dtAccel + np.asscalar(xlast[1])
    # x_new = p_last + v_last*dt + a_last*dt^2/2
    xAc = (get_accelerometer_reading(ilast, white_noise)*dtAccel**2)/2 + np.asscalar(xlast[1])*dtAccel + np.asscalar(xlast[0])
    bias = np.asscalar(xlast[2])
    return np.matrix([xAc,vAc,bias]).transpose() 


Accel_noise = []

for i in range(len(t_step)):
    Accel_noise.append(get_accelerometer_reading(i, get_white_noise_accelerometer()))


print(Accel_noise[0])

white gaussian noise of accelerometer :  [0.     0.0004]
bias of accelerometer:  [0.   0.01]
0.012728357200689815


#### Generating GPS value with noise 

In [16]:
# Functions, Variances, Means, and equations for the GPS states: Position and Velocity
dtGPS = dtAccel*40 # seconds

wmp_GPS = 0 # meters, mean of the white noise measurement of position
wmv_GPS = 0 # meters / second, mean of the white noise measurement of velocity
wVp_GPS = 1 # meter^2, variance of the white noise measurement of position
wVv_GPS = .04**2 # (4cm/s)**2 = .04**2 (m/s)^2, varaince of the white noise measurement of velocity

wNp_GPS = np.array([wmp_GPS,wVp_GPS])
wNv_GPS = np.array([wmv_GPS,wVv_GPS])


# returns the GPS noise for a given timestep
def get_measurement_noise_GPS():
    wp = np.asscalar(np.random.normal(wNp_GPS[0],np.sqrt(wNp_GPS[1]),1))
    wv = np.asscalar(np.random.normal(wNv_GPS[0],np.sqrt(wNv_GPS[1]),1))
    wb = 0 # This the the extra zero padding, corresponding to the last term (bias) of the state dynamics
    return np.matrix([wp,wv,wb]).transpose()

# [ p + n_p ]
# [ v + n_p ]
# [ 0 + bias ]
# returns the current GPS measurement vector for the system
def get_measurement_GPS(i):
    return np.matrix([true_pos[i], true_vel[i],0]).transpose() + get_measurement_noise_GPS()

GPS_noise = []
for i in range(len(t_step)):
    GPS_noise.append(get_measurement_GPS(i))

## Code Starts From HERE 

## Input Parameters

In [34]:
acc_freq = 200

# Accelerometer Noise: acc_s = acc_g + b + w 
acc_wV = 0.0004 #(m/sec^2)^2, accelerometer white noise variance
acc_bV = 0.01 #(m/sec^2)^2, accelerometer bias 

# GPS Noise
gps_pV = 10**2 # m^2
gps_vu = 100 #m/s
gps_vV = 1**2 # (m/s)^2
gps_wpV = 1 #m^2, white noise variance of gps position
gps_wvV = 0.04**2 #(m/s)^2, white noise variance of gps velocity

## Initial Conditions

In [35]:
dt = 1/acc_freq

## Prediction Step

In [45]:
A = np.array([[1, dt, -dt**2/2],[0,1,-dt],[0,0,1]])
T = -np.array([[dt**2],[dt],[0]])

P = np.array([[gps_pV, 0, 0],[0, gps_vV, 0],[0,0,acc_bV]])
Q = T*acc_wV*np.transpose(T)

def predict(state, control, P):
    # predicting the new state
    state_pred = A*state + T*control
    # predicting the new error 
    P_pred = A*P*np.transpose(A) + Q

    return state_pred, P_pred

## Correction Step

In [49]:
R = np.array([[gps_wpV, 0, 0],[0, gps_wvV, 0],[0,0,0]]) # measurement covariance matrix
H = np.matrix(np.eye(3))

def correct(state_pred, measurement, P_pred):
    S = H*P_pred*np.transpose(H) + R
    K = P_pred*np.transpose(H)*inv(S) # Kalman Gain
    state_corr = state_pred + K*(measurement-H*state_pred)
    P_corr = P_pred - K*H*P_pred
    
    return state_corr, P_corr

## Implement Kalman Filter

In [54]:
print(len(Accel_noise))
print(len(GPS_noise))


GPSbool = 0 
#Initialization
p_0 = 0 
v_0 = 0
b_0 = 0 
w_prev = 0


state = [[p_0],[v_0],[0]] # p, v, b 
control = w_prev


Kalman_pos = []
Kalman_pos.append(p_0)
for i in range(len(Accel_noise)):

    # Prediction step 
    control = get_white_noise_accelerometer()
    state_pred, P_pred = predict(state, control, P)
    state = state_pred
    P = P_pred
    
    if GPSbool == 1: # Correct if GPS exists 
        measurement = GPS[i]
        state_corr, P_corr = correct(state, measurment, P)
        state = state_corr
        P = P_corr
    
    
    Kalman_pos.append(state[0])
    Kalman_vel.append(state[1])

5999
5999


## Prediction Step 

## Correction Step